Authors : Jinsu Kim, JunHo Park

ⓒ 2022 CCNets, Inc. All Rights Reserved.

https://ccnets.org

In [1]:
import sys
path_append = "../../"
sys.path.append(path_append)  # Go up one directory from where you are.

In [2]:
# https://www.kaggle.com/datasets/ruslankl/mice-protein-expression

import os
import pandas as pd

# Define the base directory and CSV file name
base_dir = path_append + "../data/Mice Protein Expression/"  # Update this to the directory where your data folder is located
csv_file = "Data_Cortex_Nuclear.csv"  # Update this to your CSV file name if different

# Full path to the CSV file
full_path = os.path.join(base_dir, csv_file)

# Load the dataset
try:
    df = pd.read_csv(full_path)
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Failed to load data. File not found at:", full_path)

Failed to load data. File not found at: ../../../data/Mice Protein Expression/Data_Cortex_Nuclear.csv


In [3]:
df.head()

NameError: name 'df' is not defined

TrainLoader / DataLoader

In [ ]:
from tools.preprocessing.data_frame import process_dataframe
df, description = process_dataframe(df, target_columns = ["class"], drop_columns = ["MouseID"])

num_features = description['num_features']
num_classes = description['num_classes']

print("Number of features:", num_features)
print("Number of classes:", num_classes)

description

In [ ]:
from sklearn.model_selection import train_test_split
from tools.preprocessing.template_dataset import BaseDataset

train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
# Split the dataset into features and target
train_df_x = train_df.iloc[:, :-1] # all columns except the last 
train_df_y = train_df.iloc[:, -1:] # the last column

test_df_x = test_df.iloc[:, :-1] # all columns except the last
test_df_y = test_df.iloc[:, -1:] # the last column

print('train df shape: ', train_df.shape)
print('test df shape: ', test_df.shape)
trainset = BaseDataset(train_df_x, train_df_y)
testset = BaseDataset(test_df_x, test_df_y)

In [ ]:
from tools.setting.ml_params import MLParameters
from tools.setting.data_config import DataConfig
from trainer_hub import TrainerHub
import torch

data_config = DataConfig(dataset_name = 'mice_protein_expression', task_type='multi_class_classification', obs_shape=[num_features], label_size=num_classes)
#  Set training configuration from the AlgorithmConfig class, returning them as a Namespace object.
ml_params = MLParameters(ccnet_network = 'tabnet')

# Set the device to GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

# Initialize the TrainerHub class with the training configuration, data configuration, device, and use_print and use_wandb flags
trainer_hub = TrainerHub(ml_params, data_config, device, use_print=False, use_wandb=False)

In [ ]:
trainer_hub.train(trainset, testset)